In [1]:
%reset -f

import sys
sys.path.append("../data/exadata/parquet_dataset/query_tool")

import os
import pandas as pd
import numpy as np
import datetime
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.linear_model import LinearRegression
from query_tool import M100DataClient
from matplotlib import pyplot as plt
from models import *

dataset_path = "../data/m100"
processed_path = "../data/processed_m100"
client = M100DataClient(dataset_path)



In [2]:
#TODO FOR BOTH: TIMESTAMP INDICES

def extract_node_values(feature_list, n_rows, nodes_of_interest, columns, year_month):
    node_dfs = []
    for f in feature_list: 
        query_df = client.query(f, 
                      columns=columns,
                      year_month=year_month)   
        query_df.dropna(inplace=True)
        node_list = query_df["node"].to_numpy().astype(np.int32)
        for noi in nodes_of_interest:
            idxs = np.where(node_list == noi, True, False)
            nodes = query_df.iloc[idxs].iloc[:n_rows]
            node_values = nodes["value"].values
            
            node_df = pd.DataFrame(index=nodes["timestamp"])
            new_name = str(noi) + "_" + str(f)
            node_df[new_name] = node_values
            node_dfs.append(node_df)
            
    plugin_df=pd.concat(node_dfs, axis=1, join="outer")
    return plugin_df

def extract_values(feature_list, n_rows, columns, year_month):
    dfs = []
    for f in feature_list: 
        query_df = client.query(f, 
                      columns=columns,
                      year_month=year_month)   
        query_df.dropna(inplace=True)
        df = query_df.set_index("timestamp")
        df = query_df.iloc[:n_rows]
        dfs.append(df)
        
    plugin_df=pd.concat(dfs, axis=1, join="outer")
    return plugin_df

In [4]:
"""Cell purpose: pull desired features from plugins"""


# n_rows = 10000
# nodes_of_interest = [i for i in range(10)]
# # nodes_of_interest = [0]
# year_month = ["22-02"]
# 
# #ipmi collects physical/hardware properties of nodes
# ipmi_features = ["ambient", "gpu0_core_temp"]
# ipmi_columns = ["timestamp", "node", "value"]
# ipmi_df = extract_node_values(ipmi_features, n_rows, nodes_of_interest, ipmi_columns, year_month)
# ipmi_df.to_csv(os.path.join(processed_path, "ipmi.csv"))
# del ipmi_df #probably done anyway by python but no harm
# 
# #ganglia collects metrics on utilization of hardware
# ganglia_features = ["Tot"]
# ganglia_columns = ["timestamp", "node", "value"]
# ganglia_df = extract_node_values(ganglia_features, n_rows, nodes_of_interest, ganglia_columns, year_month)
# ganglia_df.to_csv(os.path.join(processed_path, "ganglia.csv"))
# del ganglia_df #probably done anyway by python but no harm
# 
# 
# #vertiv plugin collects data for AC cooling 
# vertiv_features = ["Supply_Air_Temperature"]
# vertiv_labels = ["Return_Air_Temperature"]
# vertiv_columns = ["timestamp", "value"]
# vertiv_df = extract_values(vertiv_features, n_rows, vertiv_columns, year_month)
# vertiv_df.to_csv(os.path.join(processed_path, "vertiv.csv"))
# del vertiv_df #probably done anyway by python but no harm
# 
# vertiv_labels = extract_values(vertiv_labels, n_rows, vertiv_columns, year_month)
# vertiv_labels.to_csv(os.path.join(processed_path, "vertiv_labels.csv"))
# del vertiv_labels #probably done anyway by python but no harm
# 
# 
# 
# 
# 
# 
# #weather data
# weather_features = ["temp"]
# weather_columns = ["timestamp", "value"]
# weather_df = extract_values(weather_features, n_rows, weather_columns, year_month)
# weather_df.to_csv(os.path.join(processed_path, "weather.csv"))
# del weather_df #probably done anyway by python but no harm

#schneider collects data on liquid cooling system
# schneider_df = client.query(["Tot"],
#                   columns=['timestamp', 'value'],
#                   year_month=["22-02"])

# logics collects power consumption data from equipment and devices
# logics_df = client.query(["Tot"],
#                   columns=['timestamp', "device", "panel", 'value'],
#                   year_month=["22-02"])







Retrieving data of type: float
Retrieving data of type: int32
Retrieving data of type: float
Retrieving data of type: float
Retrieving data of type: float
Retrieving data of type: float


In [5]:
weather_df = pd.read_csv(os.path.join(processed_path, "weather.csv"))
vertiv_df = pd.read_csv(os.path.join(processed_path, "vertiv.csv"))
vertiv_labels = pd.read_csv(os.path.join(processed_path, "vertiv_labels.csv"))
ganglia_df= pd.read_csv(os.path.join(processed_path, "ganglia.csv"))
ipmi_df = pd.read_csv(os.path.join(processed_path, "ipmi.csv"))
plugin_dfs = [ipmi_df, ganglia_df, weather_df, vertiv_df] #start w/ small one for now
df = pd.concat(plugin_dfs)


In [7]:
"""Cell purpose: preprocess and/or merge data as desired"""
def df_to_X_y(df, feature_list, label_list):
    return df.loc[feature_list], df.loc[label_list]
    

feature_list = ["0_ambient"]
label_list = ["0_gpu0_core_temp"] #not what we actually want, but for testing
feature_df = df[feature_list] 
label_df = df[label_list]

#align t and t+1
feature_df = feature_df.iloc[:-1]
label_df = label_df.iloc[1:]


trainX = None
trainY = None



KeyboardInterrupt: 

In [ ]:
# Will have to make work with timestamp indexign later
class M100Data(Dataset):
    def __init__(self, feature_df, label_df, transform = None, label_transform = None):
        self.feature_df = feature_df
        self.label_df = label_df
        self.transform = transform
        self.feature_dim = self.feature_df.shape[1]
        self.label_dim = self.label_df.shape[1]
    def __len__(self):
        return self.feature_df.shape[0]

    def __getitem__(self, idx):
        feature = torch.as_tensor(self.feature_df.iloc[idx].values)
        label = torch.as_tensor(self.label_df.iloc[idx].values)
        return feature, label
    
train_ds = M100Data(feature_df, label_df)
trainloader = DataLoader(train_ds, batch_size = 8, shuffle = True)

In [ ]:
epochs = 1
model = LinearNN(train_ds.feature_dim, train_ds.label_dim)
optimizer = torch.optim.SGD(model.parameters())
loss_fn = torch.nn.CrossEntropyLoss()
for e in epochs:
    train_loss = train_epoch(model, trainloader, optimizer, loss_fn, epochs)
    # eval_loss = eval_epoch(model, evalloader, loss_fn)

In [ ]:
"""Cell purpose: train"""
# model = LinearRegression().fit(merged_df["air_temp"].to_numpy().reshape(-1,1), merged_df["ambient_temp"].to_numpy().reshape(-1,1))

In [ ]:
"""Cell purpose: inference"""
# preds = model.predict(test_df["air_temp"].to_numpy().reshape(-1,1))


In [ ]:
"""Cell purpose: benchmarking"""

# labels = test_df["ambient_temp"].to_numpy().reshape(-1,1)
# 
# 
# timesteps = [t for t in range(preds.shape[0])]
# colors = iter(plt.cm.rainbow(np.linspace(0, 0.5, 2)))
# plt.scatter(preds, labels, c=["blue", "red"])
# 
# plt.legend(loc="upper left")
# plt.xlabel("Time")
# plt.ylabel("Ambient temperature")
# plt.grid()
# plt.show()



In [ ]:
""
